## Analyze A/B Test Results

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)
- [Conclusions](#conclusions)
- [Resources](#resources)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
# We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

ModuleNotFoundError: No module named 'matplotlib'

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [ ]:
df = pd.read_csv('ab_data.csv')
df.head()

b. Use the cell below to find the number of rows in the dataset.

In [ ]:
df.shape

c. The number of unique users in the dataset.

In [ ]:
df.user_id.nunique()

d. The proportion of users converted.

In [ ]:
df['converted'].mean()

e. The number of times the `new_page` and `treatment` don't match.

In [ ]:
df.query('(group=="control" & landing_page=="new_page") or (group=="treatment" & landing_page=="old_page")').user_id.count()

f. Do any of the rows have missing values?

In [ ]:
df.info()

None of the rows contain any missing values.

`2.` For the rows where **treatment** does not match with **new_page** or **control** does not match with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to figure out how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [ ]:
df2 = df.query(
    '(group=="control" & landing_page=="old_page") or (group=="treatment" & landing_page=="new_page")')
df2.head()

In [ ]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (
    df2['landing_page'] == 'new_page')) == False].shape[0]

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [ ]:
df2.user_id.nunique()

In [ ]:
df.shape

b. There is one **user_id** repeated in **df2**.  What is it?

In [ ]:
df2[df2.user_id.duplicated()]

c. What is the row information for the repeat **user_id**? 

In [ ]:
df2.query('user_id == "773192"')

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [ ]:
df2.drop_duplicates(subset='user_id', inplace=True)

`4.` Use **df2** in the cells below to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [ ]:
convert_rate = df2.converted.mean()
convert_rate

b. Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
control_convert = df2.query('group == "control"')['converted'].mean()
control_convert

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
treat_convert = df2.query('group == "treatment"')['converted'].mean()
treat_convert

d. What is the probability that an individual received the new page?

In [ ]:
df2.query('landing_page == "new_page"').count()[0] / df2['user_id'].nunique()

e. Consider your results from parts (a) through (d) above, and explain below whether you think there is sufficient evidence to conclude that the new treatment page leads to more conversions.

At this stage, the evidence indicates that the conversions are marginally higher with the control group (0.120386) than the treatment group (0.118808). Even if the converse were true, there would not be enough evidence to suggest that the treatment page would lead to more conversions. This is because factors such as change aversion and test span durations are not considered.

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

$$H_0: p_{new} \leq p_{old}$$$$H_1: p_{new} \gt p_{old}$$

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

a. What is the **conversion rate** for $p_{new}$ under the null? 

In [ ]:
p_new = convert_rate
p_new

b. What is the **conversion rate** for $p_{old}$ under the null? <br><br>

In [ ]:
p_old = convert_rate
p_old

c. What is $n_{new}$, the number of individuals in the treatment group?

In [ ]:
n_new = df2.query(('landing_page == "new_page"')).count()[0]
n_new

d. What is $n_{old}$, the number of individuals in the control group?

In [ ]:
n_old = df2.query(('landing_page == "old_page"')).count()[0]
n_old

e. Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [ ]:
new_page_converted = np.random.binomial(n=1, p=p_new, size=n_new)

f. Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [ ]:
old_page_converted = np.random.binomial(n=1, p=p_old, size=n_old)

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [ ]:
p_diff = new_page_converted.mean() - old_page_converted.mean()
p_diff

h. Create 10,000 $p_{new}$ - $p_{old}$ values using the same simulation process you used in parts (a) through (g) above. Store all 10,000 values in a NumPy array called **p_diffs**.

In [ ]:
p_diffs = []

for _ in range(10000):
    new_page_converted = np.random.binomial(n=1, p=p_new, size=n_new)
    old_page_converted = np.random.binomial(n=1, p=p_old, size=n_old)
    p_diffs.append(new_page_converted.mean() - old_page_converted.mean())

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
# converting to numpy array
p_diffs = np.array(p_diffs)

# plot histogram
plt.hist(p_diffs)
plt.title(
    "Simulated differences in conversion rates under null hypothesis", fontsize=14)
plt.xlabel('Difference in Probability', fontsize=12)

# add comparison with ab_data.csv (as calculated under Part 1 - Probabilty 4b and 4c)
plt.axvline(treat_convert - control_convert, color='r')

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
(p_diffs > (treat_convert - control_convert)).mean()

k. Please explain using the vocabulary you've learned in this course what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

Part j compared the proportion of the simulated differences in conversion rates under the null hypothesis that was greater than the difference in conversion rate calculated in 4b and 4c which was from ab_data.csv (ie. the real difference). In scientific studies this is called the p-value.

The type I error rate for this test is 0.05. A p-value less than this would indicate there is a statistical difference between the new and old page. In other words, it would be highly unlikely that we would observe the real conversion rate difference in ab_data.csv in the sampling distrubtion under the null hypothesis.

As the p_value is 0.9044, this suggests we would fail to reject the null hypothesis and conclude there is insufficient evidence to say there is a difference in conversion between the new and old pages.

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
import statsmodels.api as sm

convert_old = df2.query('converted==1 & group=="control"').user_id.count()
convert_new = df2.query('converted==1 & group=="treatment"').user_id.count()
print('convert_old:', convert_old)
print('convert_new:', convert_new)
print('n_new:', n_new)
print('n_old:', n_old)

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](https://docs.w3cub.com/statsmodels/generated/statsmodels.stats.proportion.proportions_ztest/) is a helpful link on using the built in.

In [ ]:
z_score, p_value = sm.stats.proportions_ztest(
    [convert_new, convert_old], [n_new, n_old], alternative='larger')
print(z_score, p_value)

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

The z_score is -1.31 and the p_value is 0.905. The z-critical for an upper-tailed test with $\alpha$ = 0.05 is 1.645. Since -1.31 is not greater than 1.645, this means we should not reject the null. The p_value is virtually identidcal to the p-value in part j. The results of both would allow us to reach the conclusion in part k (ie. fail to reject the null).

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

This is an example of logistic regression since we are working with only two possibles outcomes (ie. conversion or no conversion).

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives. However, you first need to create in df2 a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
df2['intercept'] = 1

df2[['control', 'treatment']] = pd.get_dummies(df2['group'])
df2.head()

In [ ]:
df2.rename(columns={'treatment': 'ab_page'}, inplace=True)
df2.drop('control', axis=1, inplace=True)
df2.head()

c. Use **statsmodels** to instantiate your regression model on the two columns you created in part b., then fit the model using the two columns you created in part **b.** to predict whether or not an individual converts. 

In [ ]:
logit_mod = sm.Logit(df2['converted'], df2[['intercept', 'ab_page']])
results = logit_mod.fit()

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
results.summary2()

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in **Part II**?

The p-value for ab_page is 0.1899 which indicates that treatment is not a useful variable for determining conversion as it is above the $\alpha$ level of 0.05. The hypthosis for ab_page variable is as follows:


$$H_0: \beta_1 = 0$$

$$H_1: \beta_1 \neq 0$$


Therefore you would reject the null hypothesis for this particular variable. This is a two-tailed test whereas in Part II it is a one-tailed test as follows:


$$H_0: p_{new} \leq p_{old}$$

$$H_1: p_{new} \gt p_{old}$$


That's why the p-values are different.

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

Considering other factors in the regression model will give you a better picture of what is influencing conversion. For example, perhaps duration spent on the page, native language of the user or time of day are relevant factors. Then you can plan your changes on these factors if there's enough statistical evidence to suggest they are a predictor for conversion.

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. You will need to read in the **countries.csv** dataset and merge together your datasets on the appropriate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [ ]:
countries_df = pd.read_csv('./countries.csv')
df_new = countries_df.set_index('user_id').join(
    df2.set_index('user_id'), how='inner')
df_new.head()

In [ ]:
df_new['country'].value_counts()

In [ ]:
df_new[['CA', 'UK', 'US']] = pd.get_dummies(df_new['country'])
df_new.tail(10)

In [ ]:
# do not fit with U.S variable as this will be the baseline
logit_mod = sm.Logit(df_new['converted'], df_new[['intercept', 'CA', 'UK']])
results = logit_mod.fit()
results.summary2()

The above results indicate that the country variables (CA and UK) have p-values greater than 0.05. Therefore there is insufficient evidence to suggest that the country variables predict the conversion rate for the two types of landing pages. Hence we would keep the old page (ie. fail to reject the null hypothesis).

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [ ]:
df_new['CA_ab_page'] = df_new['CA'] * df_new['ab_page']
df_new['UK_ab_page'] = df_new['UK'] * df_new['ab_page']
df_new.head()

In [ ]:
logit_mod = sm.Logit(df_new['converted'], df_new[[
                     'intercept', 'CA', 'UK', 'ab_page', 'CA_ab_page', 'UK_ab_page']])
results = logit_mod.fit()
results.summary2()

Again, all the the variables have p-values greater than 0.05 and therefore we would conclude there is not enough evidence to suggest that these variables can predict the conversion rate of the two pages. We would fail to reject the null hypothesis.

<a id='conclusions'></a>
## Conclusions

- There was not enough evidence to reject the null hypthesis ($H_0$). We should reject $H_1$ and keep the old page.

- The evidence from the various techniques used above all supported this conclusion. However, the analysis and results are limited only based on available data - effects of change aversion and novelty effects may influence the results. Practical considerations may also need to be factored in while making decisions.

- There was even suggestion that the old page very marginally resulted in better conversions.

- Logistic Regression also gives results that agree with the results of A/B testing. Country does not appear to be a very useful factor in the regression model. The fit of logistic regression models does not appear to be great from the Pseudo R square values - The model can be revised for better predictions.

- The duration of the experiment is 14 days. The influence of time may be a major factor in determining the conclusions. It would be good to analyze the data for different subsets of time and look at how the conversion rates differ between control and treatment groups.

<a id='resources'></a>
## Resources

- The below two resources provided a way on dealing with the problem AttributeError: module 'scipy.stats' has no attribute 'chisqprob' when running .fit.summary() for logistic regression:

http://www.statsmodels.org/stable/generated/statsmodels.discrete.discrete_model.LogitResults.summary2.html?highlight=summary2#statsmodels.discrete.discrete_model.LogitResults.summary2

https://stackoverflow.com/questions/49814258/statsmodel-attributeerror-module-scipy-stats-has-no-attribute-chisqprob

- Using LaTeX in jupyter notebooks:

https://stackoverflow.com/questions/13208286/how-to-write-latex-in-ipython-notebook

https://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Typesetting%20Equations.html

https://nbviewer.jupyter.org/github/ipython/ipython/blob/2.x/examples/Notebook/Display%20System.ipynb#LaTeX

- Many other stackoverflow posts

- Course content

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Analyze_ab_test_results_notebook.ipynb'])